In [5]:
import unicodedata

buses = []
target = 0

line_list = [line.rstrip() for line in open("Inputs/Day13.txt")]

target = int(line_list[0])
buses = [int(x) for x in line_list[1].split(',') if x.isdigit()]

print(target)
print(buses)

1003055
[37, 41, 433, 23, 17, 19, 29, 593, 13]


In [13]:
def infinite_sequence(hop):
    num = hop
    while True:
        yield num
        num += hop

generators = []
for b in buses:
    generators.append({'id': b, 'generator': infinite_sequence(b)})
    
departure = {'bus': 0, 'timestamp': target * 100}
for g in generators:
    for s in g['generator']:
        if s > target:
            if (departure['timestamp'] > s):
                departure = {'bus': g['id'], 'timestamp': s}
            break;
            
print(departure)    
dif = departure['timestamp'] - target
print(dif * departure['bus'])

{'bus': 41, 'timestamp': 1003065}
410


# Answer Part 2

I solved part 1 on my own, and got as far as figuring out that the buses are pairwise coprime, tried to derive 
a formula for figuring out the remainder but ran out of math. 

Turns out this has been done already thousands of years ago :-) https://brilliant.org/wiki/chinese-remainder-theorem/

The trick is to reformat the bus ids and positions into a list of modulos and remainders for the number we are looking for, and feed that into the Chinese Remainder Theorem

From the example:

Line 1 has modulo 7, remainder 0

Line 2 has modulo 13, remainder 1

Line 3 has modulo 59, remainder 4

Line 4 has modulo 31, remainder 6

Line 5 has modulo 19, remainder 7

Solution from here: https://github.com/bsamseth/advent-of-code/blob/master/aoc-2020/day-13/day_13.py


In [8]:
from toolz import reduce, last

with open("Inputs/Day13.txt") as f:
    timestamp = int(f.readline())
    busses = [
        (i, int(bus))
        for i, bus in enumerate(f.readline().strip().split(","))
        if bus != "x"
    ]

# Problem to solve: Find an integer x such that
#
#    x + offset_i = 0 (mod bus_id_i), for all i
# => x = -offset_i (mod bus_id_i) = bus_id_i - offset_i (mod bus_id_i)
#
# Solution given by Chinese Remainder Theorem because all the bus IDs are prime.
# Let r_i = offset_i and m_i = bus_id_i, and N = m_1 m_2 ... m_n
#
#    x = sum_i  (m_i-r_i) * N / m_i * z_i
#
# where z_i is the inverse of y_i mod m_i (that is, y_i z_i = 1 (mod m_i) ).
N = reduce(lambda a, b: a * b, map(last, busses))  # Product of all IDs (or moduli).
print("Part 2:", sum((m - r) * N // m * pow(N // m, -1, m) for r, m in busses) % N)

[(0, 37), (27, 41), (37, 433), (45, 23), (54, 17), (56, 19), (66, 29), (68, 593), (81, 13)]
Part 2: 600691418730595
